# Toronto House Price Web Scrape

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import lightgbm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import statsmodels.api as sm
import statsmodels.formula.api as smf

import math
pd.options.display.float_format = '{:,.2f}'.format

import urllib.request
import time
import bs4 as bs
#from geopy.geocoders import Nominatim
#import geopandas as gpd
import seaborn as sns
from scipy import stats
from bs4 import BeautifulSoup
from requests.exceptions import RequestException
from contextlib import closing
from requests import get
import re
import warnings
warnings.filterwarnings('ignore')



In [2]:
def download_links(url):
    req = urllib.request.Request(url,headers = {'User-Agent':user_agent})
    page = urllib.request.urlopen(req)
    front_page = BeautifulSoup(page, 'html.parser')
    #links = front_page.find_all('div', attrs={'class':'info'})
    link_lst = []
    
    for link in links:
        a = str(link).find('href=')
        b = str(link).find('-vow')
        link_lst.append(str(link)[a+6:b+4])
    
    return link_lst,front_page

In [4]:
# download the links from frontpage
links = []
for n in np.arange(1,6):
    url = 'https://www.zoocasa.com/toronto-on-sold-listings?page='+str(n)
    links += download_links(url)

In [5]:
#save the links
with open(input_dir+'Links20210221.txt', 'w') as f:
    for item in links:
        f.write("%s\n" % item)

In [6]:
#read links
links = []
with open(input_dir+'Links20210221.txt', 'r') as f:
        links.append(f.readlines())
f.close()
links = links[0]
links = [s[:-1] for s in links]


In [7]:
len(links)

125

In [8]:
def parse_data(html):
    #Extract information with class named
    info_lst = []
    
    if html.find("span",{'itemprop':'addressLocality'}) != None:
        info_lst.append(['city',html.find("span",{'itemprop':'addressLocality'}).text])
    
    if html.find("span",{'itemprop':'addressRegion'}) != None:
        info_lst.append(['region',html.find("span",{'itemprop':'addressRegion'}).text])
        
    if html.find("meta",{'itemprop':'postalCode'}) != None:
        info_lst.append(['poscode',html.find("meta",{'itemprop':'postalCode'}).get("content")])
        
    if html.find("meta",{'itemprop':'addressCountry'}) != None:
        info_lst.append(['country',html.find("meta",{'itemprop':'addressCountry'}).get("content")])
   
    #Extract the information with class
    soldprice = html.find("div",{'class':'sold-price'})
    if soldprice != None:
        if soldprice.find('span', class_='blur') != None:
            info_lst.append(['soldPrice',soldprice.find('span', class_='blur').text])
    
    listprice = html.find("div",{'class':'list-price'})
    if listprice != None:
        if listprice.find('span', class_='blur') != None:
            info_lst.append(['listPrice',listprice.find('span', class_='blur').text])
    
    bedbath = None
    if html.find("div",{'class':'beds-baths'}) != None:
        bedbath = html.find("div",{'class':'beds-baths'}).findAll('span', class_='blur')
        
    lst = ['beds','bath','sqft','parking']
    if bedbath != None:
        for n in np.arange(len(bedbath)):
            info = bedbath[n].text
            info_lst.append([lst[n],info])

    #Append description
    if html.find('p',{'class':'description padded'}) != None:
        info_lst.append(['description',str(html.find('p',{'class':'description padded'}).find('span',class_ = "blur"))[19:-7]])

    #Extract the information without span
    lst = []
    for ele in html.findAll('span'):
        if str(ele).find('span>')==1 and str(ele).find('</span>')>2:
                ele = str(ele).replace('<span>','').replace('</span>','')
                lst.append(ele)
    for var in [v for v in col_list if v in lst]:
        info_lst.append([var, lst[lst.index(var)+1]])
    
    #create dataframe
    
    dictionary = {}
    for i in range(len(info_lst)):
        dictionary[info_lst[i][0]] = [info_lst[i][1]]
    info_df = pd.DataFrame.from_dict(dictionary)
    return info_df


In [9]:
def get_details(link_lst):
    house_details = pd.DataFrame(columns=col_list)
    for link in link_lst:
        url = 'https://www.zoocasa.com' + str(link)
        req = urllib.request.Request('https://www.zoocasa.com'+link,headers = {'User-Agent':user_agent})
        page = urllib.request.urlopen(req)
        html = BeautifulSoup(page, 'lxml')
        #print(html)
        house_details = pd.concat([house_details,parse_data(html)],axis=0)
    return house_details

In [ ]:
for n in np.arange(5):
    details = get_details(links[n:n+500])
    details.to_csv(input_dir+'Link'+str(n)+'_'+str(n+500)+'.csv')

In [10]:
link = links[0]
url = 'https://condos.ca/toronto?mode=Sold&end_date_unix=relative%2C-365&page=2'
req = urllib.request.Request(url,headers = {'User-Agent':user_agent})
page = urllib.request.urlopen(req)
html = BeautifulSoup(page, 'html5lib')

In [39]:
for n in np.arange(100,2000,500):
    details = get_details(links[n:n+500])
    details.to_csv(input_dir+'Link'+str(n)+'_'+str(n+500)+'.csv')

In [4]:
df = pd.DataFrame()
for n in np.arange(0,16000,500):
    details = pd.read_csv(input_dir+'Link'+str(n)+'_'+str(n+500)+'.csv')
    df = pd.concat([df,details],axis=0)

In [5]:
df.to_csv(input_dir +'data.csv')

In [19]:
df.drop(columns = ['listprice','soldprice','Unnamed: 0'], inplace = True)

In [80]:
def str_to_num(_str):
    if type(_str) == str:
        if re.sub('[^0-9]+', '', _str) != '':
            return float(re.sub('[^0-9]+', '', _str))
        else:
            return 0
    else:
        return _str

In [160]:
df['sqft_min'] = df['sqft'].apply(lambda x:x[:x.find('–')])
df['sqft_max'] = df['sqft'].apply(lambda x:x[x.find('–')+1:x.find('sq')])
df['zip'] = df['poscode'].apply(lambda x:str(x)[:3])
df['num_beds'] = df['beds'].apply(lambda x:str_to_num(x[:x.find('+')]) + str_to_num(x[x.find('+')+1:x.find('beds')])*0.6)

In [13]:
col_list =   ['city',
'region',
 'poscode',
 'country',
 'soldprice',
 'listprice',
 'beds',
 'baths',
 'sqft',
 'parking',
 'comments',
 'MLS® Number',
 'Type',
 'Levels',
 'Size',
 'Maintenance Fees',
 'Taxes',
 'Locker',
 'Ensuite Laundry',
 'Balcony',
 'Furnished',
 'Laundry Level',
 'Exposure',
 'Exterior',
 'Garage',
 'Approx. Age',
 'Stories',
 'Pets',
 'Corp Number',
 'Elevator',
 'Concierge',
 'Exercise Room',
 'Indoor Pool',
 'Party/Meeting Room',
 'Rooftop Deck/Garden',
 'Visitor Parking',
 'Hydro',
 'Water',
 'Building Insurance',
 'Condo Taxes',
 'Cable TV',
 'Central Air',
 'Parking Maintenance',
 'A/C',
 'Heating Fuel',
 'Central Vac',
 'Fireplace',
 'Acreage',
 'Lot Size',
 'Basement',
 'Driveway',
 'Garage Spaces',
 'Guest Suites',
 'Media Room',
 'Bbqs Allowed',
 'Gym',
 'Bike Storage',
 'Recreation Room',
 'Security System',
 'Outdoor Pool',
 'Security Guard']

In [8]:
pd.DataFrame(col_list,columns=['var'])

,var
0,city
1,region
2,poscode
3,country
4,soldprice
...,...
56,Bike Storage
57,Recreation Room
58,Security System
59,Outdoor Pool
